In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [0]:
path_1 = "/content/drive/My Drive/BERT_Anadi/Data Produced/bert_train_embed_1000_docs.pickle"
path_2 = "/content/drive/My Drive/BERT_Anadi/Data Produced/bert_gs_embed_all.pickle"


data = pd.read_pickle((path_1))
# data = data.drop(columns = ['doc_id'])
X_pred = (pd.read_pickle(path_2))

# gc.collect()

In [3]:
data.shape

(130759, 1026)

In [4]:
X_pred.shape

(9092, 1025)

In [5]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test
X_pred.drop(columns = ['doc_id'], inplace = True)
del data, target
gc.collect()

0

In [6]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(91531, 1024) (91531,) (39228, 1024) (39228,)


In [7]:
print(y_train.value_counts()+y_test.value_counts())

0.0    103205
1.0     27554
Name: importance, dtype: int64


# Start Model Fitting From Here

### Model 1

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import tensorflow as tf

x_train = tf.keras.utils.normalize(X_train, axis=1)
x_test = tf.keras.utils.normalize(X_test, axis=1)
print("Normalization Complete!")

model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Flatten())   # input layer
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))   # output layer

model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
print("Model Ready")

model.fit(x_train, np.array(y_train), epochs=6, validation_data = (x_test, np.array(y_test)))

Normalization Complete!
Model Ready
Epoch 1/6

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2861/2861 [==============================] - 22s 8ms/step - loss: 0.5025 - accuracy: 0.7893 - val_loss: 0.5019 - val_accuracy: 0.7888
Epoch 2/6
2861/2861 [==============================] - 22s 8ms/step - loss: 0.4898 - accuracy: 0.7895 - val_loss: 0.5188 - val_accuracy: 0.7888
Epoch 3/6
2861/2861 [==============================] - 21s 7ms/step - loss: 0.4807 - accuracy: 0.7895 - val_loss: 0.4993 - val_accuracy: 0.7888
Epoch 4/6
2861/2861 [===================

In [9]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []
acc = []

pred = (model.predict(X_test))

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = cutoff, y = f1, mode='lines', name='f1'))
fig.add_trace(go.Scatter(x = cutoff, y = acc, mode='lines', name='accuracy'))
fig.show()

In [13]:
max_cutoff = cutoff[f1 == max(f1)]
print(max(f1), max_cutoff)

0.4684692504665597 [0.315]


# View Results

In [16]:
GS_sent = pd.read_pickle("/content/drive/My Drive/BERT_Anadi/Data Produced/gs_sent.pickle")
GS_sent.columns = ["Sents", "doc_id"]


predictions = model.predict(X_pred) >= max_cutoff
probability_of_predictions = model.predict(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions == 1])
result['probabilities'] = probability_of_predictions[predictions == 1]
result['doc_id'] = result['doc_id']

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

,Sents,doc_id,probabilities
0,three civil appeals stemming three revision pe...,0.0,0.333663
1,high court deciding various issues remanded ca...,0.0,0.405143
2,shri achutananda purohit appellant intermediar...,0.0,0.325074
3,estate vested state april force act crucial qu...,0.0,0.405169
14,directly concerned issue compensation dealt ea...,0.0,0.389739
...,...,...,...
9077,upon review decided cases survey occasions whe...,49.0,0.388706
9082,firstly writ certiorari exercise original juri...,49.0,0.343628
9083,secondly writ certiorari record proceedings ce...,49.0,0.405057
9084,exercise supervisory jurisdiction high court m...,49.0,0.404948


In [0]:
result.to_csv("/content/drive/My Drive/BERT_Anadi/result_roberta_best.csv")